# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
#TODO: push to github
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine 

import pickle

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.multioutput import MultiOutputClassifier

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine) #table name same as db name
X = df['message']#TODO: message column
Y = df.iloc[:, 4:]#TODO: 36 categories

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
categories_labels = Y.columns
Y.nunique()

related-1                   2
request-0                   2
offer-0                     2
aid_related-0               2
medical_help-0              2
medical_products-0          2
search_and_rescue-0         2
security-0                  2
military-0                  2
child_alone-0               1
water-0                     2
food-0                      2
shelter-0                   2
clothing-0                  2
money-0                     2
missing_people-0            2
refugees-0                  2
death-0                     2
other_aid-0                 2
infrastructure_related-0    2
transport-0                 2
buildings-0                 2
electricity-0               2
tools-0                     2
hospitals-0                 2
shops-0                     2
aid_centers-0               2
other_infrastructure-0      2
weather_related-0           2
floods-0                    2
storm-0                     2
fire-0                      2
earthquake-0                2
cold-0    

In [8]:
Y['related-1'].unique()

array([1, 0])

In [9]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26180 entries, 0 to 26179
Data columns (total 36 columns):
related-1                   26180 non-null int64
request-0                   26180 non-null int64
offer-0                     26180 non-null int64
aid_related-0               26180 non-null int64
medical_help-0              26180 non-null int64
medical_products-0          26180 non-null int64
search_and_rescue-0         26180 non-null int64
security-0                  26180 non-null int64
military-0                  26180 non-null int64
child_alone-0               26180 non-null int64
water-0                     26180 non-null int64
food-0                      26180 non-null int64
shelter-0                   26180 non-null int64
clothing-0                  26180 non-null int64
money-0                     26180 non-null int64
missing_people-0            26180 non-null int64
refugees-0                  26180 non-null int64
death-0                     26180 non-null int64
other_aid

In [10]:
#convert df back to numpy array for convenience used of sklearn prediction, calssification report
Y = Y.values

In [11]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):    
    #normalize
    ##lower case
    text = text.lower() 
    ##remove puntuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    
    #tokenize text
    words = word_tokenize(text)
    
    #lemmatize
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in text]
        
    
    return lemmed


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
#TODO: speed up
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))), #TODO: test n_jobs=-1 efficiency
])
 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = pipeline.predict(X_test)
print(y_pred)


[[1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]


In [17]:
type(y_test)

numpy.ndarray

In [18]:
#display classification report for each class label 0-35
#pay attension on COLUMN, child_alone-0, since all elements belongs to the same class value (Class Imbalance)

for i in range(y_test.shape[1]):
    print(categories_labels[i])
    print(classification_report(y_test[:,i], y_pred[:,i]))
    print()

related-1
             precision    recall  f1-score   support

          0       0.47      0.29      0.36      1489
          1       0.81      0.90      0.85      5056

avg / total       0.73      0.76      0.74      6545


request-0
             precision    recall  f1-score   support

          0       0.85      0.98      0.91      5393
          1       0.64      0.20      0.30      1152

avg / total       0.81      0.84      0.80      6545


offer-0
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6509
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6545


aid_related-0
             precision    recall  f1-score   support

          0       0.65      0.80      0.72      3858
          1       0.58      0.38      0.46      2687

avg / total       0.62      0.63      0.61      6545


medical_help-0
             precision    recall  f1-score   support

          0       0.93   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
#TODO: make some plot on class balance

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
#TODO:
parameters = {
    'n_estimators': [100, 300, 500, 800, 1200],
    'max_depth': [5, 8, 15, 25, 30]    
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# TODO: basic metrics
# TODO: Advance: micro/macro average(recap)

#test with GridSearchCV
######cv.fit(X_train, y_train)
######y_pred = cv.predict(X_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#TODO: add more features, preprocessing such as normalisation, stopwords removal(as well as to speed up training since data size decreases)
#TODO: feature union to include new features

### 9. Export your model as a pickle file

In [20]:
#export pickle
with open('pipeline.pickle', 'wb') as f:
    pickle.dump(pipeline, f)

#double check
with open('pipeline.pickle', 'rb') as f:
    pipeline = pickle.load(f)
    print(pipeline.predict(X_test[0:10]))

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.